In [5]:
pwd

'c:\\Users\\Soomin Choi\\Desktop\\카이스트\\금융빅데이터분석\\팀프'

In [ ]:
import time
import random
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import json
import re
import os

# 댓글을 리스트로 저장하는 함수
def get_comments_list(driver):
    comments_list = []
    actions = ActionChains(driver)

    while True:
        try:
            # '더 보기' 버튼이 나타날 때까지 대기
            show_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.show_more.comment_header'))
            )
            actions.move_to_element(show_more_button).click().perform()
            time.sleep(random.uniform(0.5, 1.0))  # 무작위 대기
        except:
            break

    comment_items = driver.find_elements(By.CSS_SELECTOR, 'li.fdb_itm.clear')
    for comment in comment_items:
        try:
            comment_text = comment.find_element(By.CSS_SELECTOR, 'div.xe_content').text
            comments_list.append(comment_text)
        except:
            continue

    return comments_list

# 데이터 저장 함수
def save_to_json(data, filename):
    try:
        # 기존 파일이 있으면 읽어와서 업데이트
        if os.path.exists(filename):
            with open(filename, "r", encoding="utf-8") as file:
                existing_data = json.load(file)
        else:
            existing_data = []

        # 데이터 추가 및 저장
        existing_data.append(data)
        with open(filename, "w", encoding="utf-8") as file:
            json.dump(existing_data, file, ensure_ascii=False, indent=4)
        print(f"데이터가 파일에 저장되었습니다: {data['ID']}")
    except Exception as e:
        print(f"파일 저장 중 오류 발생: {e}")

# WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
filename = "real_time_data.json"

# 현재 날짜와 -7일까지 설정
today = datetime.now() - timedelta(days=0)
end_date = today - timedelta(days=7)
page = 1

while True:
    driver.get(f"https://theqoo.net/hot?page={page}")
    print(f"페이지 {page} 크롤링 중...")

    post_elements = WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'table.bd_lst tbody.hide_notice tr'))
    )

    for post_element in post_elements:
        try:
            if 'notice' in post_element.get_attribute('class'):
                continue

            title_element = post_element.find_elements(By.CSS_SELECTOR, '.title a')
            if not title_element:
                continue

            title = title_element[0].text
            post_url = title_element[0].get_attribute("href")

            post_date_text = post_element.find_element(By.CSS_SELECTOR, '.time').text

            if ":" in post_date_text:
                post_date = today
            elif len(post_date_text) == 8:
                post_date = datetime.strptime(post_date_text, '%y.%m.%d')
            else:
                post_date = datetime.strptime(post_date_text, '%m.%d').replace(year=today.year)

            if post_date < end_date:
                driver.quit()
                print("크롤링이 완료되었습니다.")
                exit()

            category = post_element.find_element(By.CSS_SELECTOR, '.cate').text if post_element.find_elements(By.CSS_SELECTOR, '.cate') else "N/A"
            views_text = post_element.find_element(By.CSS_SELECTOR, '.m_no').text.replace(",", "")
            views = int(views_text) if views_text.isdigit() else 0
            comment_count = post_element.find_elements(By.CSS_SELECTOR, '.far.fa-comment-dots')
            comments_count = int(comment_count[0].text) if comment_count and comment_count[0].text.isdigit() else 0

            driver.execute_script("window.open(arguments[0]);", post_url)
            driver.switch_to.window(driver.window_handles[-1])

            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.rhymix_content.xe_content')))
            content = driver.find_element(By.CSS_SELECTOR, 'div.rhymix_content.xe_content').text

            comments = get_comments_list(driver)

            post_data = {
                "ID": "theqoo-" + re.search(r'(?<=hot/)\d+', post_url).group(),
                "community": "theqoo",
                "category": category,
                "title": title,
                "post_date": post_date.strftime('%Y-%m-%d %H:%M'),
                "view": views,
                "comments_count": len(comments),
                "link": post_url,
                "content": content,
                "comments": comments
            }

            # 한 게시물 크롤링 후 바로 저장
            save_to_json(post_data, filename)

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(random.uniform(0.8, 1.5))

        except Exception as e:
            print(f"오류 발생: {e}")
            if "429" in str(e):
                print("서버 요청 제한 발생(429), 10초 대기 중...")
                time.sleep(10)

    page += 1

# WebDriver 종료
driver.quit()

페이지 8 크롤링 중...
11.09
데이터가 파일에 저장되었습니다: theqoo-3478387963
11.09
데이터가 파일에 저장되었습니다: theqoo-3478377164
11.09
오류 발생: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x00F9EBD3+24307]
	(No symbol) [0x00F28D74]
	(No symbol) [0x00E0C323]
	(No symbol) [0x00DFCA18]
	(No symbol) [0x00DFC939]
	(No symbol) [0x00E0E790]
	(No symbol) [0x00E88EE9]
	(No symbol) [0x00E6FC26]
	(No symbol) [0x00E4219C]
	(No symbol) [0x00E4311D]
	GetHandleVerifier [0x01248D93+2818227]
	GetHandleVerifier [0x012A542E+3196750]
	GetHandleVerifier [0x0129D9D2+3165426]
	GetHandleVerifier [0x0103DA70+675216]
	(No symbol) [0x00F31B3D]
	(No symbol) [0x00F2EA18]
	(No symbol) [0x00F2EBB5]
	(No symbol) [0x00F21640]
	BaseThreadInitThunk [0x76067BA9+25]
	RtlInitializeExceptionChain [0x7727C0CB+107]
	RtlClearBits [0x7727C04F+191]

오류 발생: Message: disconnected: not connected to DevTools

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x00F9EBD3+24307]
	(No symbol) [0x00F28D74]
	(No symbol) [0x00E0C323]
	(No symbol) [0x00DFCA18]
	(No symbol) [0x00DFC939]
	(No symbol) [0x00E0E790]
	(No symbol) [0x00E88EE9]
	(No symbol) [0x00E6FC26]
	(No symbol) [0x00E4219C]
	(No symbol) [0x00E4311D]
	GetHandleVerifier [0x01248D93+2818227]
	GetHandleVerifier [0x012A542E+3196750]
	GetHandleVerifier [0x0129D9D2+3165426]
	GetHandleVerifier [0x0103DA70+675216]
	(No symbol) [0x00F31B3D]
	(No symbol) [0x00F2EA18]
	(No symbol) [0x00F2EBB5]
	(No symbol) [0x00F21640]
	BaseThreadInitThunk [0x76067BA9+25]
	RtlInitializeExceptionChain [0x7727C0CB+107]
	RtlClearBits [0x7727C04F+191]


In [ ]:
import time
import random
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import json
import re
import os

# 댓글을 리스트로 저장하는 함수
def get_comments_list(driver):
    comments_list = []
    actions = ActionChains(driver)

    while True:
        try:
            show_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.show_more.comment_header'))
            )
            actions.move_to_element(show_more_button).click().perform()
            time.sleep(random.uniform(0.5, 1.0))
        except:
            break

    comment_items = driver.find_elements(By.CSS_SELECTOR, 'li.fdb_itm.clear')
    for comment in comment_items:
        try:
            comment_text = comment.find_element(By.CSS_SELECTOR, 'div.xe_content').text
            comments_list.append(comment_text)
        except:
            continue

    return comments_list

# 데이터 저장 함수
def save_to_json(data, filename):
    try:
        if os.path.exists(filename):
            with open(filename, "r", encoding="utf-8") as file:
                existing_data = json.load(file)
        else:
            existing_data = []

        existing_data.append(data)
        with open(filename, "w", encoding="utf-8") as file:
            json.dump(existing_data, file, ensure_ascii=False, indent=4)
        print(f"데이터가 파일에 저장되었습니다: {data['ID']}")
    except Exception as e:
        print(f"파일 저장 중 오류 발생: {e}")

# WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
filename = "real_time_data.json"

today = datetime.now() - timedelta(days=2)
end_date = today - timedelta(days=7)
page = 45

while True:
    driver.get(f"https://theqoo.net/hot?page={page}")
    print(f"페이지 {page} 크롤링 중...")

    post_elements = WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'table.bd_lst tbody.hide_notice tr'))
    )

    for post_element in post_elements:
        try:
            if 'notice' in post_element.get_attribute('class'):
                continue

            title_element = post_element.find_elements(By.CSS_SELECTOR, '.title a')
            if not title_element:
                continue

            title = title_element[0].text
            post_url = title_element[0].get_attribute("href")

            # 게시물 목록에서 post_date 텍스트 가져오기
            post_date_text = post_element.find_element(By.CSS_SELECTOR, '.time').text

            # 목록에서 날짜 변환
            if ":" in post_date_text:
                post_date_list = today
            elif len(post_date_text) == 8:
                post_date_list = datetime.strptime(post_date_text, '%y.%m.%d')
            else:
                post_date_list = datetime.strptime(post_date_text, '%m.%d').replace(year=today.year)

            # 날짜 범위 검사: 목록의 post_date가 end_date보다 과거면 크롤링 중단
            if post_date_list < end_date:
                print("크롤링 종료: 게시물이 end_date보다 과거입니다.")
                driver.quit()
                exit()

            # 날짜 범위가 유효하지 않으면 다음 게시물로 이동
            if not (end_date <= post_date_list <= today):
                continue

            category = post_element.find_element(By.CSS_SELECTOR, '.cate').text if post_element.find_elements(By.CSS_SELECTOR, '.cate') else "N/A"
            views_text = post_element.find_element(By.CSS_SELECTOR, '.m_no').text.replace(",", "")
            views = int(views_text) if views_text.isdigit() else 0
            comment_count = post_element.find_elements(By.CSS_SELECTOR, '.far.fa-comment-dots')
            comments_count = int(comment_count[0].text) if comment_count and comment_count[0].text.isdigit() else 0

            # 개별 게시물로 이동하여 정확한 post_date 가져오기
            driver.execute_script("window.open(arguments[0]);", post_url)
            driver.switch_to.window(driver.window_handles[-1])

            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.rhymix_content.xe_content')))
            content = driver.find_element(By.CSS_SELECTOR, 'div.rhymix_content.xe_content').text

            try:
                # 개별 게시물에서 정확한 post_date 가져오기
                post_date_text = driver.find_element(By.CSS_SELECTOR, 'div.side.fr span').text
                post_date = datetime.strptime(post_date_text, '%Y.%m.%d %H:%M')
            except Exception as e:
                print(f"게시물 날짜 가져오기 오류: {e}")
                post_date = post_date_list

            comments = get_comments_list(driver)

            post_data = {
                "ID": "theqoo-" + re.search(r'(?<=hot/)\d+', post_url).group(),
                "community": "theqoo",
                "category": category,
                "title": title,
                "post_date": post_date.strftime('%Y-%m-%d %H:%M'),
                "view": views,
                "comments_count": len(comments),
                "link": post_url,
                "content": content,
                "comments": comments
            }

            save_to_json(post_data, filename)

            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(random.uniform(0.8, 1.5))

        except Exception as e:
            print(f"오류 발생: {e}")
            if "429" in str(e):
                print("서버 요청 제한 발생(429), 10초 대기 중...")
                time.sleep(10)

    page += 1

driver.quit()


페이지 45 크롤링 중...
데이터가 파일에 저장되었습니다: theqoo-3470265296
데이터가 파일에 저장되었습니다: theqoo-3470260512
데이터가 파일에 저장되었습니다: theqoo-3470251323
데이터가 파일에 저장되었습니다: theqoo-3470249410
데이터가 파일에 저장되었습니다: theqoo-3470239590
데이터가 파일에 저장되었습니다: theqoo-3470239433
데이터가 파일에 저장되었습니다: theqoo-3470206061
데이터가 파일에 저장되었습니다: theqoo-3470203799
데이터가 파일에 저장되었습니다: theqoo-3470197802
데이터가 파일에 저장되었습니다: theqoo-3470186649
데이터가 파일에 저장되었습니다: theqoo-3470162970
데이터가 파일에 저장되었습니다: theqoo-3470136033
데이터가 파일에 저장되었습니다: theqoo-3470104321
데이터가 파일에 저장되었습니다: theqoo-3470068886
데이터가 파일에 저장되었습니다: theqoo-3470063074
데이터가 파일에 저장되었습니다: theqoo-3470051940
데이터가 파일에 저장되었습니다: theqoo-3470042780
데이터가 파일에 저장되었습니다: theqoo-3470042709
데이터가 파일에 저장되었습니다: theqoo-3470032868
데이터가 파일에 저장되었습니다: theqoo-3470029830
페이지 46 크롤링 중...
데이터가 파일에 저장되었습니다: theqoo-3470032868
데이터가 파일에 저장되었습니다: theqoo-3470029830
데이터가 파일에 저장되었습니다: theqoo-3470008149
데이터가 파일에 저장되었습니다: theqoo-3470002412
데이터가 파일에 저장되었습니다: theqoo-3469987949
데이터가 파일에 저장되었습니다: theqoo-3469981490
데이터가 파일에 저장되었습니다: theqoo-3469929

MaxRetryError: HTTPConnectionPool(host='localhost', port=64716): Max retries exceeded with url: /session/3c115a76348d99fc976b852ce7390d98/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000029ECABC1490>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

: 